In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Import libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn

In [ ]:
df = pd.read_csv(os.path.join(dirname,filename))

In [ ]:
df.head()

Check for missing values

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

Some EDA

In [ ]:
df["Drug"].value_counts() # Drug B and Drug C are under represented

In [ ]:
df.describe()

Distribution of ages

In [ ]:
sns.set(font_scale=1)
plt.title("Distribution of Age")
sns.distplot(df["Age"])

Age

In [ ]:
plt.figure(figsize=(12,8))
plt.title("Boxplot for ages")
sns.boxplot(x="Drug",y="Age",data=df)

It appears that the Drug B is for higher age group

In [ ]:
df.groupby("Drug").mean()["Age"].sort_values(ascending=False)

Sex

In [ ]:
df["Sex"].value_counts()

In [ ]:
plt.figure(figsize=(12,8))
plt.title("Drug wise Gender Distribution")
sns.countplot(x="Sex",hue="Drug",data=df)

Boxplot 

In [ ]:
plt.figure(figsize=(12,10))
plt.title("Boxplot of Age and Distribution")
sns.boxplot(x="Sex",y="Age",data=df)

Mean Age of Male is higher

In [ ]:
df.head()

One hot Encode Function

In [ ]:
def one_hot_encode(dataframe, col):
    dummy = pd.get_dummies(dataframe[col],drop_first=True)
    dummy.columns = [str(i)+"_dumy_"+col for i in range(len(dummy.columns))]
    dataframe = dataframe.join(dummy)
    dataframe.drop([col],axis=1,inplace=True)
    return dataframe

In [ ]:
df = one_hot_encode(df,"Sex")

In [ ]:
df.head()

BP

In [ ]:
plt.figure(figsize=(12,10))
plt.title("Boxplot of Blood Pressure")
sns.boxplot(x="BP",y="Age",data=df)


In [ ]:
#One hot Encode the BP variable
df = one_hot_encode(df,"BP")

In [ ]:
df.head()

Na_to_K

In [ ]:
sns.distplot(df["Na_to_K"])
#There seems to be some outliers present

In [ ]:
sns.boxplot(df["Na_to_K"])

Drugwise Boxplot

In [ ]:
plt.figure(figsize=(12,9))
sns.boxplot(x="Drug",y="Na_to_K",data=df)

Querying outliers

In [ ]:
df[df.Na_to_K>32]

As the outliers are for the same drug Y, I will keep it in the dataset

In [ ]:
plt.figure(figsize=(12,10))
sns.scatterplot(x="Age",y="Na_to_K",data=df,hue="Drug",s=500,alpha=0.7)

Drug Y and Drug B seems to be from a stronger family

Cholestrol

In [ ]:
df["Cholesterol"].value_counts()

In [ ]:
df = one_hot_encode(df,"Cholesterol")

In [ ]:
df.head()

# Label Encode the Drug Column

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
lb = LabelEncoder()

In [ ]:
df["Drug"] = lb.fit_transform(df["Drug"])

In [ ]:
df.head()

Label classes

In [ ]:
lb.classes_

In [ ]:
df.columns

In [ ]:
numerical_feats = df[["Age",'Na_to_K']]

In [ ]:
numerical_feats

Separting features and labels

In [ ]:
X = df.drop(["Drug"],axis=1).values
y = df["Drug"].values

Performing a train test split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test, y_train,y_test = train_test_split(X,y,test_size=15,random_state=42,stratify=y)

Using MinMaxScaler

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled_X_train = scaler.fit_transform(X_train)
scaled_X_test = scaler.transform(X_test)

Using K neigbhor Classifiers

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

Training the model

In [ ]:
knn = KNeighborsClassifier(n_neighbors=1, p=2 )
knn.fit(scaled_X_train,y_train)

Performing prediction on validation dataset

In [ ]:
pred_k = knn.predict(scaled_X_test)

Function to evaluate results

In [ ]:
from sklearn import metrics
def evaluate_result(true,pred):
    print(metrics.classification_report(true,pred))
    print(metrics.accuracy_score(true,pred))

In [ ]:
evaluate_result(y_test,pred_k)

Doing a prediction on whole dataset

In [ ]:
sc = MinMaxScaler()
scaled_X = sc.fit_transform(X)

In [ ]:
predictions = knn.predict(scaled_X)

Combining the result with the original dataframe

In [ ]:
df=df.join(pd.DataFrame(predictions,columns=["predictions"]))

In [ ]:
df[["Drug","predictions"]]

In [ ]:
evaluate_result(df.Drug,df.predictions)

Actual labels versus Datapoints

In [ ]:
plt.figure(figsize=(15,8))
sns.scatterplot(x='Age',y="Na_to_K",hue="Drug",data=df,s=500,alpha=0.5)

Precitions vs datapoint

In [ ]:
plt.figure(figsize=(15,8))
sns.scatterplot(x='Age',y="Na_to_K",hue="predictions",data=df,s=500,alpha=0.5)

# Thank you for visiting this notebook, please upvote if adds to your knowledge (: